In [6]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
import json
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.authorities.personalName import PersonalName
from api.src.function.authorities.personalName.create_graph import MakePersonalGraph
from api.src.schemas.authorities.mads import Authority, EditVariant
from api.src.function.authorities.personalName.editPersonalName import EditAuthorityPersonalName
from api.src.function.authorities.personalName.variantPersonalName import editVariantPersonalName, addVariantPersonalName
from api.src.function.authorities.edit_uri import DelMads, PostMads
from api.src.function.authorities.personalName.editPersonalName import EditAuthorityPersonalName, EditFullerName, DeleteFullerName, PostFullerName
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib.plugins.sparql import prepareQuery
from urllib.error import HTTPError
from api.src.function.authorities.personalName.docPersonalName import MakeDocPersonalName
from api.src.schemas.authorities.mads import Uri

In [7]:
obj = {
  "id": "n80002329",
  "uri": "http://id.worldcat.org/fast/48407",
  "mads": "hasCloseExternalAuthority",
  "collection": "name"
}
request = Uri(**obj)
request

Uri(id='n80002329', uri='http://id.worldcat.org/fast/48407', mads='hasCloseExternalAuthority', collection='name')

In [8]:
child = request.uri.split("/")[-1]
child

'48407'

In [2]:
obj = {
  "id": "n80002329",
  "label": "Machado de Assis",
  "date": "1839-1908",
  "birthDate": "1839-06-21",
  "birthPlace": "Rio de Janeiro (Brazil)",
  "deathDate": "1908-09-29",
  "fullerName": "Joaquim Maria Machado",
  "hasCloseExternalAuthority": ["http://id.worldcat.org/fast/48407", "http://www.wikidata.org/entity/Q311145"],
  "hasExactExternalAuthority": ["http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept"],
  "hasVariant": [
    {
      "value": "Machado de Assis, Joaquín María",
      "type": "PersonalName",
      "date": "1839-1908"
    },
    {
      "value": "Mashado de Assiz, Zhoakin",
      "type": "PersonalName",
      "date": "1839-1908"
    },
    {
      "value": "Assis, Machado de",
      "type": "PersonalName",
      "date": "1839-1908"
    }
  ],
  "hasAffiliation": [
    {
    "organization": "http://id.loc.gov/authorities/names/n82059239",
    "affiliationStart": "1897",
    "affiliationEnd": "1908"
    }
  ],
  "occupation": [
    "http://id.loc.gov/authorities/subjects/sh85034154",
    "http://id.loc.gov/authorities/subjects/sh85092863"
  ]
}

request = PersonalName(**obj)
request

PersonalName(recordChangeDate='2023-04-13T15:29:52', id='n80002329', label='Machado de Assis', date='1839-1908', birthDate='1839-06-21', birthPlace='Rio de Janeiro (Brazil)', deathDate='1908-09-29', fullerName='Joaquim Maria Machado', hasCloseExternalAuthority=['http://id.worldcat.org/fast/48407', 'http://www.wikidata.org/entity/Q311145'], hasExactExternalAuthority=['http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept'], hasVariant=[Authority(value='Machado de Assis, Joaquín María', lang=None, date='1839-1908', type='PersonalName'), Authority(value='Mashado de Assiz, Zhoakin', lang=None, date='1839-1908', type='PersonalName'), Authority(value='Assis, Machado de', lang=None, date='1839-1908', type='PersonalName')], hasAffiliation=[Affiliation(organization='http://id.loc.gov/authorities/names/n82059239', affiliationStart='1897', affiliationEnd='1908')], occupation=['http://id.loc.gov/authorities/subjects/sh85034154', 'http://id.loc.gov/authorities/subjects/sh85092863'], contribut

In [3]:
doc = MakeDocPersonalName(request)
doc

{'recordChangeDate': '2023-04-13T15:29:52',
 'id': 'n80002329',
 'label': 'Machado de Assis',
 'date': '1839-1908',
 'birthDate': '1839-06-21',
 'birthPlace': 'Rio de Janeiro (Brazil)',
 'deathDate': '1908-09-29',
 'fullerName': 'Joaquim Maria Machado',
 'hasCloseExternalAuthority': [{'id': 'n80002329!48407',
   'collection': 'worldcat',
   'label': 'Machado de Assis, 1839-1908',
   'uri': 'http://id.worldcat.org/fast/48407'},
  {'id': 'n80002329!Q311145',
   'collection': 'wikidata',
   'label': 'Machado de Assis',
   'uri': 'http://www.wikidata.org/entity/Q311145',
   'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Machado_de_Assis_aos_57_anos.jpg'}],
 'hasExactExternalAuthority': ['http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept'],
 'hasVariant': ['Machado de Assis, Joaquín María',
  'Mashado de Assiz, Zhoakin',
  'Assis, Machado de'],
 'hasAffiliation': [{'id': 'n80002329!n82059239',
   'uri': 'http://id.loc.gov/authorities/names/n82059239',
   'label': 'Ac

In [ ]:
uri = {'id': 'n80002329!48407',
   'collection': 'worldcat',
   'label': 'Machado de Assis, 1839-1908',
   'uri': 'http://id.worldcat.org/fast/48407'}

doc = {'id': request.id,
        f'{request.mads}': {"add": uri}  }

In [10]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":146}}\n'

In [5]:
def GetLabelLoc(uri):
    url = uri.replace('http', 'https')
    g = Graph()
    try:
        g.parse(f'{url}.madsrdf.nt', format='nt')
    except HTTPError as e:
        return uri

    q = f"""
    SELECT ?label WHERE {{
        <{uri}> madsrdf:authoritativeLabel ?label }}"""
    query = prepareQuery(q, initNs={"madsrdf": "http://www.loc.gov/mads/rdf/v1#"})
    [label] = [result.label.value for result in g.query(query)]
    return label

In [6]:
label = GetLabelLoc('http://id.loc.gov/authorities/subjects/sh85034154')
label

'Critics'

In [8]:
if request.occupation:
    for i in request.occupation:
        label = GetLabelLoc(i)
        print(label)

Critics
Novelists


In [28]:
if request.hasAffiliation:
    affilations = list()
    for i in request.hasAffiliation:
        label = GetLabelLoc(i.organization)
        affilation = {
            'uri': i.organization,
            'organization': label,
            'start': i.affiliationStart,
            'end': i.affiliationEnd            
        }
        affilations.append(affilation)

        print(label)

Academia Brasileira de Letras


NameError: name 'GetLabelLoc' is not defined

In [31]:
if request.hasAffiliation:
    affilations = list()
    for i in request.hasAffiliation:
        i

In [5]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":117}}\n'

In [22]:
url = 'http://id.loc.gov/authorities/names/n82059239.json'
g = Graph()
g.parse('https://id.loc.gov/authorities/names/n82059239.madsrdf.nt', format='nt')



<Graph identifier=Na9078afd6aa346978127758dc620eb91 (<class 'rdflib.graph.Graph'>)>

In [23]:
g.serialize('institution.ttl', format='ttl')

<Graph identifier=Na9078afd6aa346978127758dc620eb91 (<class 'rdflib.graph.Graph'>)>

In [25]:
id = 'n82059239'

q = f"""
    SELECT ?label WHERE {{
        <http://id.loc.gov/authorities/names/{id}> madsrdf:authoritativeLabel ?label }}"""

query = prepareQuery(q, initNs={"madsrdf": "http://www.loc.gov/mads/rdf/v1#"})
[label] = [result.label.value for result in g.query(query)]
label

'Academia Brasileira de Letras'